## 트리의 앙상블 (이진분류 할필요 x )

In [1]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델들 입니다
# 앙상블 모델들은 결정트리(Decision Tree)를 기반으로 만들어 졌습니다
# 앙상블 모델들
# - 랜덤포레스트(Random Forest)
# - 엑스트라 트리(Extra Trees)
# - 그래디언트 부스팅(Gradient Boosting)
# - 히스토그램 기반 그래디언트 부스팅(Histogram-base Gradient Boosting)

## 랜덤포레스트(Random Forest)

In [2]:
### - 앙상블 모델 중에 가장 대표격 모델로 사용됨
### - 안정적인 성능으로 널리 사용
### - 앙상블 모델 중에 가장 먼저 시도하는 모델
### - 훈련데이터에서 과대적합되는 것을 막아준다
### - 검증 데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

### 학습개념 
### - 결정트리를 하나하나를 랜덤하게 만들어서 숲을 만들기
### - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후 다시 원본 훈련데이터에반환
### - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음 (중복허용)

### 부트스트랩 샘플(Bootstrap Sample)
### - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링 하는 방식
### - 샘플 추출방식
###   1. 원본에서 랜덤샘플 추출
###   2.훈련 이후 사용이 끝나면 원본에 반환
###   3.다시 원본에서 샘플 추출, 이때 중복 값 추출될수도있다
#### - 위 순사를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용하고있다 


### **** 랜덤포레스트는 교차검증을 허용 ****

In [3]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
wine = pd.read_csv('./data/08_wine.csv')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data,target, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

# 표준화(정규화) 작업도 해주면 좋다 

(4872, 3) (4872,)
(1625, 3) (1625,)


### 훈련모델 생성하기

In [5]:
# 랜덤포레스트 패키지 : sklearn.ensemble
# 랜덤포레스트 클래스(모델) : RandomForestClassifier
# 교차검증 패키지 : sklearn.model_selection
# 교차검증 : cross_validate()
# 교차검증 후 훈련검증 결과와 테스트검증결과 확인하기

# 랜덤포레스트 객체생성 : 코어 모두 사용
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs= -1 , random_state=42)

# 교차검증 진행
# - return_train_score : 검증결과 반환받기
from sklearn.model_selection import cross_validate
scores = cross_validate(rfc,train_input, train_target, return_train_score = True, n_jobs=-1)

# 최종 훈련평가 결과 및 검증결과
print(scores)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))


{'fit_time': array([0.25937247, 0.25969505, 0.26253867, 0.26479316, 0.2518723 ]), 'score_time': array([0.03181005, 0.03842092, 0.03265023, 0.03263855, 0.03037333]), 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
0.997844759088341 0.8914208392565683


In [6]:
rfc.fit(train_input , train_target)
## 특성중요도 조회하기
# 랜덤포레스트 여러가지 골고루 사용 
print(rfc.feature_importances_)

[0.23155241 0.49706658 0.27138101]


In [7]:
### 기능 사용
# 훈련에 참여하지 못한 잔여샘플 사용하는 기능 
#기본은 사용X
rfc = RandomForestClassifier(oob_score =True, n_jobs = -1 , random_state=42)
rfc.fit(train_input, train_target)
print(rfc.oob_score_)
                             

0.8981937602627258


### 엑스트라 트리(Extra Tree)

In [8]:
# - 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련함
# - 랜덤포레스트와의 차이점
#   : 부트스트랩 샘플링을 지원하지 않음
#    : 훈련 데이터 전체를 이용하여 결정트리를 생성
#   : 무작위로 트리를 분리함
# - 사용되는 속성 :splitter = 'random' 무작위 속성
# - 장점 
#   : 과대적합을 막고, 검증데이터의 평가 값을 높일 수 있음
#   : 특성 대이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 분순도등 여러가지 조건에 따라 결정 트리를 생성하기 떄문에 속도가 느린 바면
# - 엑스트라트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점 

In [9]:
### 사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreesClassifier

### 코어 전체 사용, train 및 test 결과값 출력
### 교차검증 train 및 test 결과 확인

from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(n_jobs= -1 , random_state=42)

from sklearn.model_selection import cross_validate
scores = cross_validate(etc,train_input, train_target,return_train_score=True,n_jobs=-1)

# 최종 훈련평가 결과 및 검증 결과
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.997844759088341 0.8903937240035804


In [10]:
etc.fit(train_input, train_target)
print(etc.feature_importances_)

[0.20702369 0.51313261 0.2798437 ]


## 그래디언트 부스팅 ( Gradient Boosting)

In [11]:
# 깊이 (max_depth)가 얕은 결정트리를 사용
# - 기본적으로 max_depth =3 을 사용
# - 결정트리는 100개 사용
### *** 기존에 다른 훈련모델의 결과가 좋지않을때 사용하는 모델 ****
# 기존 훈련모델의 오차를 많이 보완
# 성능 향상을 위한 모델로 주로 사용
# 과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강함

# 성능향상 테스트 방법
# - 결정트리의 갯수를 조절하면서 테스트 진행 
# - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
#   : 기본 학습률은 0.1

# 단점
# - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에 훈련속도는 느림
# - 이런 느린 속도를 개선한 모델이 ' 히스토그램 기반 그래디언트 부스팅' 모델 

## 그래디언트 부스팅 모델생성

In [12]:
### 사용하는 클래스 (모델) : GradientBoostingClassifier
# 객체 생성시 아무것도 안주고 seed 값만 줍니다
# 교차검증시에는 train, test 결과값 출력
from sklearn.ensemble import GradientBoostingClassifier
gdc = GradientBoostingClassifier(random_state=42)

from sklearn.model_selection import cross_validate
scores = cross_validate(gdc,train_input, train_target,return_train_score=True,n_jobs=-1)

# 최종 훈련평가 결과 및 검증 결과
print(np.mean(scores['train_score']),np.mean(scores['test_score']))


# 0.997844759088341 0.8914208392565683   : 랜덤포레스트
# 0.997844759088341 0.8903937240035804   : 엑스트라 트리

0.8894704231708938 0.8715107671247301


In [13]:

gdc.fit(train_input, train_target)
print(gdc.feature_importances_)

[0.12517641 0.73300095 0.14182264]


In [14]:
gdc.score(test_input,test_target)

0.8578461538461538

## 학습률 적용하기

In [15]:
# 학습률이 커지면 트리 보정을 강하게 하기때문에 복잡한 모델을 만들어서 일반화 성능을 떨어뜨리게된다
# 학습률 : learning_rate=0.1 기본값

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
gdc = GradientBoostingClassifier(n_estimators=100,
                                 learning_rate=0.1,
                                 random_state=42)

from sklearn.model_selection import cross_validate
scores = cross_validate(gdc,train_input, train_target,return_train_score=True,n_jobs=-1)
scores
# 최종 훈련평가 결과 및 검증 결과
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8894704231708938 0.8715107671247301


## 히스토그램 기반 그래디언트 부스팅 
####  - Histogram-base Gradient Boosting

In [17]:
## 사용하는 클래스(모델) : HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
hgbc = HistGradientBoostingClassifier(random_state=42)
scores= cross_validate(hgbc,train_input, train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9380129799494501 0.8805410414363187


In [19]:
hgbc.fit(train_input, train_target)
hgbc.score(test_input, test_target)

0.8584615384615385

## sklearn이외 다른 패키지에서 지원하는
## 히스토그램 기반 그래디언트 부스팅 기능 모델들

## XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores=cross_validate(xgb, train_input, train_target,
                     return_train_score=True, n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

## LightGBM

In [ ]:
# - a마이크로스프트에서 만든 히스토그램 기반 그래디언트 부스트 패키지
# - 훈련속도가 매우빠름
# - 최신 기술을 많이 적용하고 있어서 인기가 올라가고있음 

In [ ]:
# 아나콘다 사용시 : conda install -c conda-forge lightgbm
# 파이썬 사용시 : pip install lightgbm
from lightgbm import LGBMClassifier

lgb = XGBClassifier(random_state=42)
scores=cross_validate(lgb, train_input, train_target,
                     return_train_score=True, n_jobs=-1)

scores
print(np.mean(scores['train_score']),np.mean(scores['test_score']))